In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%tensorflow_version 2.x
!pip install tensorflow-gpu==2.0.0-rc0
!pip install h5py==2.9.0

In [0]:
import os
import sys
import numpy as np
import tensorflow as tf

In [4]:
tf.__version__

'2.0.0-rc0'

In [0]:
sys.path.append('drive/My Drive/pix2pix_project/') # Incluimos los modulos alamcenados de mi drive

In [0]:
# Importamos todos los modulos de mi drive
from dataset import load, load_image_train, load_image_test
#from models import Discriminator, Generator
from pix2pix_models import Discriminator, Generator
from training import train_step, train, generator_optimizer, discriminator_optimizer, generate_images, plt

In [0]:
PATH = './drive/My Drive/pix2pix_project'
PATH_IN = PATH + '/input_images_'
PATH_OUT = PATH + '/output_images_'
SUFIJOS = ['a', 'b'] # nos va servir para cargar los dataset ambas partes del problema.

In [0]:
datasets = [] 
datatests = []
# obtenemos las urls del dataset de mi drive
for i in SUFIJOS:
    file_names = [ i+'/'+arch.name for arch in os.scandir(PATH_IN+i) if arch.is_file()]
    n = len(file_names)
    train_n = round(n * 0.80)
    #partes
    train_urls = file_names[:train_n]
    np.random.shuffle(train_urls)
    datasets.append(train_urls)
    test_urls = file_names[train_n:n]
    datatests.append(test_urls)

In [0]:
BUFFER_SIZE = 400
BATCH_SIZE = 1

In [10]:
# cargamos el dataset_a
train_dataset_a = tf.data.Dataset.from_tensor_slices(datasets[0])
train_dataset_a = train_dataset_a.map(load_image_train)
train_dataset_a = train_dataset_a.cache().shuffle(BUFFER_SIZE)
train_dataset_a = train_dataset_a.batch(BATCH_SIZE)
# cargamos el dataset_b
train_dataset_b = tf.data.Dataset.from_tensor_slices(datasets[1])
train_dataset_b = train_dataset_b.map(load_image_train)
train_dataset_b = train_dataset_b.cache().shuffle(BUFFER_SIZE)
train_dataset_b = train_dataset_b.batch(BATCH_SIZE)

# los datos de validación_a
test_dataset_a = tf.data.Dataset.from_tensor_slices(datatests[0])
test_dataset_a = test_dataset_a.map(load_image_test)
test_dataset_a = test_dataset_a.batch(BATCH_SIZE)

# los datos de validación_b
test_dataset_b = tf.data.Dataset.from_tensor_slices(datatests[1])
test_dataset_b = test_dataset_b.map(load_image_test)
test_dataset_b = test_dataset_b.batch(BATCH_SIZE)
        

In [0]:
generators = [Generator("generator_a"), 
              Generator("generator_b")]

discriminators = [Discriminator("discriminator_a"),
                  Discriminator("discriminator_b")]

In [12]:
# Entrenar solo uno de los modelo. Para entrenar el otro hay que reiniciar el entorno.
train(train_dataset_a, 2, generators[0], discriminators[0], PATH + '/checkpoints_fit' )

Se cargo el generator_a.
Se cargo el discriminator_a.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Tiempo necesario para la época: 1 es 156.60275220870972 sec
Tiempo necesario para la época: 2 es 45.7084641456604 sec


In [0]:
train(train_dataset_b, 301, generators[1], discriminators[1], PATH + '/checkpoints_fit' )

In [16]:
contador = 0
for inp, re in test_dataset_b:
    generate_images(generators[1], inp, re)
    contador += 1
    if contador == 10:
        break

Output hidden; open in https://colab.research.google.com to view.